In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22

# Koala 7B loading in 8bit on a T4

In [2]:
!nvidia-smi

Tue Apr 18 11:41:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
import textwrap

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
!nvidia-smi

Tue Apr 18 11:46:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    26W /  70W |   8151MiB / 15360MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import textwrap

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

## Run it as a HF model

In [7]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
What are the difference between Llamas, Alpacas and Koalas?')
print(wrap_text_preserve_newlines(output[0]['generated_text']))


BEGINNING OF CONVERSATION: USER: What are the difference between Llamas, Alpacas and Koalas?
Llamas, alpacas, and koalas are all unique animals with their own distinct characteristics. Here's a
comparison of these three species:

1.   Size: Llamas are larger than both alpacas and koalas. They can weigh up to 250 pounds (113 kilograms)
compared to alpacas that typically weigh around 100-150 pounds (45-68 kilograms), and koalas that weigh around
70 pounds (32 kilograms).
2.   Diet: Llamas primarily eat grasses and other vegetation, while alpacas consume mostly hay and other plant
material. Koalas mainly feed on eucalyptus leaves, which they chew into small pieces before swallowing them.
3.   Habitat: Llamas are native to South America and have been domesticated for thousands of years by
indigenous peoples in the Andes Mountains. They thrive at high altitudes where there is less competition from
predators such as pumas and eagles. Alpacas were also domesticated in the Andes Mountains but 

In [8]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: \
Write a short note to Sam Altman giving reasons to open source GPT-4')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: Write a short note to Sam Altman giving reasons to open source GPT-4.

Dear Sam,

I hope this email finds you well. I am writing to share my thoughts on the potential benefits of opening up
GPT-4 as an open-source project. As someone who has been following your work closely and is interested in the
development of AI technologies, I believe that there are several compelling reasons why it would be beneficial
to make GPT-4 available for free use by anyone.

Firstly, making GPT-4 an open-source project could help to accelerate its adoption and widespread use. By
making the code and models freely available, more people will have access to them and can build upon them,
leading to new innovations and applications. This could also lead to increased collaboration between different
organizations and individuals working on similar projects, which could ultimately result in even better
outcomes.

Secondly, opening up GPT-4 as an open-source project could help to e

In [9]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: What is the capital of England? \n')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: What is the capital of England?
Answer: The capital city of England is London.
CPU times: user 2.04 s, sys: 0 ns, total: 2.04 s
Wall time: 2.04 s


In [10]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: Write a story about a Koala playing pool and beating all the camelids.')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: Write a story about a Koala playing pool and beating all the camelids.
Once upon a time, in a far-off land where animals could talk, there lived a koala named Koko. Koko was known
for his love of playing pool and his exceptional skills at it. One day, he decided to challenge all the other
animals in the kingdom to a game of pool.

The first animal he challenged was a camel named Cammy. Cammy accepted the challenge and they played a match.
At first, Koko thought he had an easy win as he had been practicing for weeks. But to his surprise, Cammy
proved to be a formidable opponent. The two played several games back and forth, with neither one able to gain
the upper hand.

Next up was a llama named Lola. She too accepted the challenge and they played a match. Once again, Koko
thought he would have an easy win but Lola's quick reflexes and strategic thinking made her a tough opponent.
They played several games back and forth, with both players giving their be

In [27]:
%%time 
output = pipe('BEGINNING OF CONVERSATION: USER: genere une description de produit pour des nike air jordan 1, la description doit etre en français')

print(wrap_text_preserve_newlines(output[0]['generated_text']))

BEGINNING OF CONVERSATION: USER: genere une description de produit pour des nike air jordan 1, la description
doit etre en français.

Description du produit : Nike Air Jordan 1 "Bred" - Chaussures de basket-ball blanc avec des bandes noires sur
les côtés et un logo "JORDAN" sur le bas. Elles sont conçues pour être confortables et légères, avec une
structure solide qui vous aidera à atteindre vos performances optimales. Les chaussures sont fabriquées avec
des matériaux hautement performants pour vous offrir une bonne prise et une bonne tenue. Elles sont également
disponibles dans différentes tailles pour vous aider à trouver votre taille juste.
CPU times: user 28.9 s, sys: 91.9 ms, total: 29 s
Wall time: 33.1 s
